In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.path import Path
import pprint

# ------------ Load Layout and Sensor Data ------------
with open("layout.json", "r") as f:
    layout_data = json.load(f)

polygon_coords = layout_data["verts"]
room_boxes     = layout_data["room_category"]  # { room_name: [ [x1,y1,x2,y2], ... ], ... }
min_x, min_y   = layout_data["bbox"]["min"]
max_x, max_y   = layout_data["bbox"]["max"]

with open("phase1_sensors.json", "r") as f:
    sensor_data = json.load(f)
sensor_positions = np.array(sensor_data["sensors"])  # each is [x, y, r]

# ------------ Activity Levels Prompt ------------
time_blocks     = ["Morning", "Afternoon", "Evening", "Night"]
activity_levels = {t: {} for t in time_blocks}

print("Enter activity levels (0.0–1.0):")
for t in time_blocks:
    print(f"\n– {t}")
    for room in room_boxes:
        while True:
            try:
                v = float(input(f"  {room}: "))
                if 0.0 <= v <= 1.0:
                    activity_levels[t][room] = v
                    break
            except ValueError:
                pass
            print("   → Enter a number between 0.0 and 1.0.")
print("\nFinal activity levels:")
pprint.pprint(activity_levels)

# ------------ Prepare Grid & Room Labels ------------
grid_res = 0.2
xs = np.arange(min_x, max_x, grid_res)
ys = np.arange(min_y, max_y, grid_res)
xx, yy = np.meshgrid(xs, ys)
GRID   = np.c_[xx.ravel(), yy.ravel()]

# label each grid point by the room it belongs to
room_label_map = np.full(len(GRID), "", dtype=object)
for room, boxes in room_boxes.items():
    for x1, y1, x2, y2 in boxes:
        mask = (GRID[:,0] >= x1)&(GRID[:,0] <= x2)&(GRID[:,1] >= y1)&(GRID[:,1] <= y2)
        room_label_map[mask] = room

# ------------ Activation Evaluation ------------
def evaluate_mask(mask, tkey):
    cov = np.zeros(len(GRID))
    for i, on in enumerate(mask):
        if not on: continue
        x0,y0,r = sensor_positions[i]
        d = np.hypot(GRID[:,0]-x0, GRID[:,1]-y0)
        cov += (d <= r).astype(float)
    w = np.array([activity_levels[tkey].get(room,0) for room in room_label_map])
    return (cov>0).dot(w), mask.sum()

def optimize_for(tkey, trials=500):
    best_s, best_m = -1, None
    N = len(sensor_positions)
    for _ in range(trials):
        m = np.random.rand(N)>0.5
        s,_ = evaluate_mask(m, tkey)
        if s>best_s:
            best_s, best_m = s, m.copy()
    return best_m

# ------------ Run & Save Phase 2 Results ------------
results = {}
for t in time_blocks:
    mask = optimize_for(t)
    results[t] = mask.tolist()

with open("phase2_results.json","w") as f:
    json.dump(results, f, indent=2)

# ------------ Visualization per Time Block ------------
for t in time_blocks:
    mask = np.array(results[t])
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_aspect("equal")
    ax.set_title(f"Active Sensors – {t}")

    # draw floor fill and outline
    ax.add_patch(Polygon(polygon_coords, closed=True, facecolor="lightgray", zorder=0))
    ax.add_patch(Polygon(polygon_coords, closed=True, fill=False, edgecolor="black", linewidth=2, zorder=4))

    # label each room (skip 'Toilet' to avoid overlap)
    for room, boxes in room_boxes.items():
        if room.lower() == 'toilet':
            continue
        for x1,y1, x2,y2 in boxes:
            xc, yc = (x1+x2)/2, (y1+y2)/2
            ax.text(xc, yc, room, ha="center", va="center", fontsize=8, color="black", zorder=5)

    # draw sensors
    for i, on in enumerate(mask):
        x0,y0,r = sensor_positions[i]
        c = "green" if on else "red"
        ax.add_patch(plt.Circle((x0,y0), r, color=c, alpha=0.3, zorder=2))
        ax.plot(x0, y0, marker="o", color=c, zorder=3)

    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_y, max_y)
    ax.grid(True, zorder=1)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    plt.tight_layout()
    plt.show()1
    1
    